# Public Pension Plan Analysis with SQL

This notebook explores the Public Plans Database (PPD), which contains pension plan data from 2001 to 2017 across 180 public plans. The goal is to demonstrate SQL proficiency through data validation, trend analysis, and actuarial insights relevant to public retirement systems like Oregon PERS.

Key skills showcased:
- Data cleaning and validation
- Time-series and comparative analysis
- Documentation of rules and logic
- Pension funding and investment performance

dataset is the PPD_PlanLevel.csv from https://www.kaggle.com/datasets/meepbobeep/us-public-pensions-data-fiscal-years-20012016

In [ ]:
# Install dependencies
%pip install sqlalchemy

In [2]:
# Load CSV
import pandas as pd
df = pd.read_csv('PPD_PlanLevel.csv', low_memory=False)

In [3]:
# Create engine and write to database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///ppd.db')
df.to_sql('public_plans', con=engine, index=False, if_exists='replace')

3044

---
## Dataset Overview

We’ll begin by inspecting the structure of the dataset and identifying key columns for analysis.

In [4]:
# Preview the first 10 rows using pandas and SQLAlchemy
query = "SELECT * FROM public_plans LIMIT 10;"
pd.read_sql(query, con=engine)

,ppd_id,PlanName,fy,system_id,PlanFullName,source_PlanBasics,InPFS,FiscalYearType,PlanInceptionYear,PlanClosed,...,HFTotal_Trgt,COMDTotal_Rtrn,COMDTotal_Actl,COMDTotal_Trgt,CashTotal_Rtrn,CashTotal_Actl,CashTotal_Trgt,OtherTotal_Rtrn,OtherTotal_Actl,OtherTotal_Trgt
0,1,Alabama ERS,2001,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
1,1,Alabama ERS,2002,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
2,1,Alabama ERS,2003,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
3,1,Alabama ERS,2004,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
4,1,Alabama ERS,2005,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
5,1,Alabama ERS,2006,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
6,1,Alabama ERS,2007,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
7,1,Alabama ERS,2008,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
8,1,Alabama ERS,2009,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None
9,1,Alabama ERS,2010,1.0,Employees’ Retirement System of Alabama,None,1.0,1.0,1945.0,0.0,...,None,None,None,None,None,None,None,None,None,None


---
## Data Quality Audit

We’ll identify plans with missing or incomplete fiscal year data to assess coverage and consistency.

In [5]:
# Count fiscal years per plan
query = """
SELECT 
  PlanName, 
  COUNT(DISTINCT fy) AS years_reported
FROM 
  public_plans
GROUP BY 
  PlanName
ORDER BY 
  years_reported ASC;
"""
pd.read_sql(query, con=engine)

,PlanName,years_reported
0,Colorado State and School,4
1,Duluth Teachers,14
2,Alabama ERS,17
3,Alabama Teachers,17
4,Alameda County ERS,17
...,...,...
175,West Virginia PERS,17
176,West Virginia Teachers,17
177,Wichita ERS,17
178,Wisconsin RS,17


---
## Funding Ratio Trends by State

We’ll calculate the average actuarial funded ratio by state from 2010 to 2017.

In [6]:
# Average funded ratio by state
query = """
SELECT 
  StateAbbrev,
  ROUND(AVG(ActFundedRatio_GASB), 2) AS avg_funded_ratio
FROM 
  public_plans
WHERE 
  fy BETWEEN 2010 AND 2017
  AND ActFundedRatio_GASB IS NOT NULL
GROUP BY 
  StateAbbrev
ORDER BY 
  avg_funded_ratio DESC;
"""
pd.read_sql(query, con=engine)


,StateAbbrev,avg_funded_ratio
0,DC,1.02
1,SD,0.98
2,WA,0.96
3,WI,0.94
4,TN,0.93
5,NC,0.93
6,ID,0.87
7,OR,0.85
8,OK,0.84
9,ME,0.84


---
## Contribution Compliance

We’ll identify plans that consistently paid less than 100% of their required contributions.

In [7]:
# Plans with underpaid contributions
query = """
SELECT 
  PlanName,
  COUNT(*) AS underpaid_years
FROM 
  public_plans
WHERE 
  PercentReqContPaid < 100
GROUP BY 
  PlanName
HAVING 
  underpaid_years >= 3
ORDER BY 
  underpaid_years DESC;
"""
pd.read_sql(query, con=engine)


,PlanName,underpaid_years
0,Wyoming Public Employees,17
1,Wichita ERS,17
2,West Virginia Teachers,17
3,West Virginia PERS,17
4,Washington Teachers Plan 2/3,17
...,...,...
173,Arizona State Corrections Officers,13
174,Anchorage Police and Fire,12
175,Atlanta ERS,10
176,New Orleans ERS,4


---
## Investment Performance

We’ll analyze long-term investment returns to evaluate plan sustainability.

In [8]:
# Investment return analysis
query = """
SELECT 
  PlanName,
  InvestmentReturn_5yr,
  InvestmentReturn_10yr,
  InvestmentReturn_20yr
FROM 
  public_plans
WHERE 
  fy = 2017
  AND InvestmentReturn_5yr IS NOT NULL
ORDER BY 
  InvestmentReturn_10yr DESC;
"""
pd.read_sql(query, con=engine)


,PlanName,InvestmentReturn_5yr,InvestmentReturn_10yr,InvestmentReturn_20yr
0,Pittsburgh Police,0.085009,0.126799,NaN
1,Atlanta ERS,0.091545,0.075279,NaN
2,Oklahoma Teachers,0.110100,0.068722,NaN
3,Ohio Teachers,0.100600,0.068000,NaN
4,Texas Municipal,0.074600,0.067100,NaN
...,...,...,...,...
165,Lexington-Fayette County Police and Fire,0.057413,0.034215,NaN
166,Louisiana Municipal Police,0.077824,0.032954,NaN
167,Indiana PERF,0.057000,0.029000,NaN
168,Indiana Teachers,0.057000,0.029000,NaN


---
## Conclusion

This analysis demonstrates how SQL can be used to uncover meaningful insights from public pension data. By querying the dataset directly, we were able to:

- **Assess data completeness** by identifying plans with limited fiscal year reporting, which may indicate gaps in data collection or reporting practices.
- **Compare funding health across states**, revealing significant variation in average actuarial funded ratios between 2010 and 2017.
- **Highlight contribution shortfalls**, pinpointing plans that consistently paid less than 100% of their required contributions—an important signal of long-term fiscal stress.
- **Evaluate investment performance**, showing how long-term returns vary across plans and may impact financial sustainability.

These findings illustrate the power of SQL for data auditing, trend analysis, and performance benchmarking in public finance. This notebook serves as a foundation for further exploration, including visual analytics, predictive modeling, and policy evaluation.